In [48]:
import pandas as pd
import json
import requests
API_key="051d30521795e4d2a16a29de43033af3"
!ls data/zippedData | grep sv > file_list.txt

In [49]:
# get the list of tmdb movie id's

list_movie_ids=[]
list_errors=[]
with open('data/movie_ids_12_09_2021.json', 'r', encoding='utf-8') as f:

    for line in f:
        id_json=f.readline()#.strip()
        try:
#            list_movie_ids.append(json.loads(id_json))
            movie_id_json=json.loads(id_json)
        except:
            list_errors.append(line)
        else:
            list_movie_ids.append(movie_id_json)


In [50]:
query = 'https://api.themoviedb.org/3/genre/movie/list?api_key='+API_key+'&language=en-US'
response =  requests.get(query)

pd.DataFrame(response.json()['genres']).to_csv("data/zippedData/tmdb_genres.csv.gz", encoding='utf-8', compression='gzip')

In [51]:

list_movie_ids[1:3]

[{'adult': False,
  'id': 25449,
  'original_title': 'New World Disorder 9: Never Enough',
  'popularity': 0.907,
  'video': False},
 {'adult': False,
  'id': 2,
  'original_title': 'Ariel',
  'popularity': 9.857,
  'video': False}]

In [52]:
# adding tmdb_movie_ids.csv.gz to zippedData folder

pd.DataFrame(list_movie_ids).to_csv("data/zippedData/tmdb_movie_ids.csv.gz", encoding='utf-8', compression='gzip')



In [53]:
# get genre_info and save it in zippedData as gzip

query = 'https://api.themoviedb.org/3/genre/movie/list?api_key='+API_key+'&language=en-US'
response =  requests.get(query)

pd.DataFrame(response.json()['genres']).to_csv("data/zippedData/tmdb_genres.csv.gz", encoding='utf-8', compression='gzip')



In [61]:
# loading all files into a dictionary

with open ('file_list.txt', 'r') as f:
    file_list= f.readlines()
    
csv_dict = {}

for csv in file_list:
    file='data/ZippedData/'+csv.strip()
    df_name=csv.strip()
    df_name=df_name.split('.csv.gz')[0]
    if 'csv' in file:
        df=pd.read_csv(file, compression='gzip')
    elif 'tsv' in file:
        df=pd.read_csv(file, compression='gzip', delimiter='\t', encoding='latin-1')
    else: print(file, ": unkown file!")
    csv_dict[df_name] = df

csv_dict.keys()


dict_keys(['bom.movie_gross', 'imdb.name.basics', 'imdb.title.akas', 'imdb.title.basics', 'imdb.title.crew', 'imdb.title.principals', 'imdb.title.ratings', 'rt.movie_info.tsv.gz', 'rt.reviews.tsv.gz', 'tmdb.movies', 'tmdb_genres', 'tmdb_imdb_gross', 'tmdb_movie_ids', 'tn.movie_budgets'])

In [67]:
# creating a list of in house tmdb id's:

#id_list=list(csv_dict.get("tmdb.movies")["id"])
#id_list=list(set(id_list))
#len(id_list)

id_list=list(csv_dict.get("tmdb_movie_ids")["id"])
len(set(id_list))

331097

In [88]:
# Define function to get tmdb details (budget,revenue, corresponding imdb id)


def get_data(API_key, Movie_ID):
    query = 'https://api.themoviedb.org/3/movie/'+Movie_ID+'?api_key='+API_key+'&language=en-US'
    response =  requests.get(query)
    if response.status_code==200: 

        array = response.json()
        return array
    else:
        return ("error")

Sample_MovieID="10138"

get_data(API_key, Sample_MovieID).keys()

#get_data(API_key, Sample_MovieID)["genres"]

dict_keys(['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count'])

In [86]:

def get_genre (my_dict):
    id_list=[genre['id'] for genre in my_dict]
    return id_list

  
get_genre(get_data(API_key, Sample_MovieID)["genres"])       

[12, 28, 878]

In [69]:
# get genre_info and save it in zippedData as gzip

query = 'https://api.themoviedb.org/3/genre/movie/list?api_key='+API_key+'&language=en-US'
response =  requests.get(query)

pd.DataFrame(response.json()['genres']).to_csv("data/zippedData/tmdb_genres.csv.gz", encoding='utf-8', compression='gzip')



In [89]:
## Uncomment this section to download imdb_id/tmdb_id information as well as revenue/budget information from API:
tmdb_list=[]
tmdb_dic={}
for num,movie_id in enumerate(id_list):
    movie_json=get_data(API_key, str(movie_id))
    if movie_json != "error":
        tmdb_dict = {"id":movie_json["id"], "tconst":movie_json["imdb_id"], "title":movie_json["title"], "release_date":movie_json["release_date"], 
                    "revenue": movie_json["revenue"], "budget": movie_json["budget"],"vote_average":movie_json["vote_average"],
                    "popularity":movie_json["popularity"], "vote_count":movie_json["vote_count"], 
                     "runtime":movie_json["runtime"], "genres":get_genre(movie_json["genres"]) }
        tmdb_list.append(tmdb_dict)
#        if num == 5:
#            break


pd.DataFrame(tmdb_list).to_csv("data/zippedData/tmdb_imdb_gross_full.csv.gz", encoding='utf-8', compression='gzip')


In [90]:
tmdb_list

[{'id': 6124,
  'tconst': 'tt0011436',
  'title': 'Peter Voss, Thief of Millions',
  'release_date': '1921-01-01',
  'revenue': 0,
  'budget': 0,
  'vote_average': 0.0,
  'popularity': 0.6,
  'vote_count': 0,
  'runtime': 420,
  'genres': [12]},
 {'id': 25449,
  'tconst': '',
  'title': 'New World Disorder 9: Never Enough',
  'release_date': '2008-12-08',
  'revenue': 0,
  'budget': 0,
  'vote_average': 6.0,
  'popularity': 1.4,
  'vote_count': 3,
  'runtime': 69,
  'genres': []},
 {'id': 2,
  'tconst': 'tt0094675',
  'title': 'Ariel',
  'release_date': '1988-10-21',
  'revenue': 0,
  'budget': 0,
  'vote_average': 6.8,
  'popularity': 9.654,
  'vote_count': 154,
  'runtime': 73,
  'genres': [18, 80, 35]},
 {'id': 5,
  'tconst': 'tt0113101',
  'title': 'Four Rooms',
  'release_date': '1995-12-09',
  'revenue': 4257354,
  'budget': 4000000,
  'vote_average': 5.7,
  'popularity': 18.985,
  'vote_count': 2122,
  'runtime': 98,
  'genres': [80, 35]},
 {'id': 8,
  'tconst': 'tt0825671',
  '